# GOAL
Identify non professional TalentCards users in GBQ.

# PACKAGES

In [42]:
import pandas as pd
from google.oauth2 import service_account
import pandas_gbq
import yaml

# PARAMETERS

In [2]:
project_id = "analytics-dev-308300"

credentials = service_account.Credentials.from_service_account_file(
    "../keys/gcp_key.json",
)

# FUNCTIONS

# DATA WRANGLING

In [27]:
df_non_pro = pd.read_excel('../data/in/20210829_talentcards_non_professional_users.xlsx')
df_non_pro = df_non_pro[['Identifier']]
df_non_pro['Identifier']=df_non_pro['Identifier'].replace({'-':''},regex=True)
df_non_pro['Flagged']=True
df_non_pro

,Identifier,Flagged
0,AQGJEGFMZPFF,True
1,AQZWKMAQHJFL,True
2,AQCAGKPUKWUI,True
3,AQAISHKLLSMM,True
4,AQNCCPYYWMAN,True
5,AQSNUTGKIPXA,True
6,AQOJUJAYOCKL,True
7,AQBJIIYALBBE,True
8,AQHQBUDACZMX,True
9,AQQOKIWJESYZ,True


In [19]:
query = """
SELECT * FROM dtm_engagement.dim_users
"""

df_bq_users=pd.read_gbq(query=query, credentials=credentials, project_id=project_id)

In [20]:
df_bq_users.head()

,user_id,group_id,access_token,email,first_name,last_name,mobile,created_at,last_login,updated_at,date_str,date_str_sp,extraction_date,last_login_date
0,20029,1811,AQAFCTCXCVLE,jmbenedetto@byhumane.com,JM,Benedetto,+5511957828747,2021-07-25T00:11:04+00:00,None,2021-08-26 12:06:52+00:00,2021-08-29 22:00:07,2021-08-29 19:00:08,2021-08-29,None
1,20029,1818,AQZSDRJJNPOF,jmbenedetto@byhumane.com,JM,Benedetto,+5511957828747,2021-07-25T00:11:04+00:00,None,2021-08-26 12:06:52+00:00,2021-08-29 22:00:07,2021-08-29 19:00:08,2021-08-29,None
2,20029,1861,AQWPWSQTCCJZ,jmbenedetto@byhumane.com,JM,Benedetto,+5511957828747,2021-07-25T00:11:04+00:00,None,2021-08-26 12:06:52+00:00,2021-08-29 22:00:07,2021-08-29 19:00:08,2021-08-29,None
3,20030,1811,AQFRTURQSSHF,ajacomy@byhumane.com,Aurelien,Jacomy,None,2021-07-25T02:42:03+00:00,None,2021-08-27 18:04:40+00:00,2021-08-29 22:00:07,2021-08-29 19:00:08,2021-08-29,None
4,20030,1818,AQBJIIYALBBE,ajacomy@byhumane.com,Aurelien,Jacomy,None,2021-07-25T02:42:03+00:00,None,2021-08-27 18:04:40+00:00,2021-08-29 22:00:07,2021-08-29 19:00:08,2021-08-29,None


In [38]:
df_bq_users_flagged=df_bq_users.merge(df_non_pro,left_on='access_token',right_on='Identifier').drop(columns='Identifier')

In [39]:
df_bq_users_flagged

,user_id,group_id,access_token,email,first_name,last_name,mobile,created_at,last_login,updated_at,date_str,date_str_sp,extraction_date,last_login_date,Flagged
0,20030,1818,AQBJIIYALBBE,ajacomy@byhumane.com,Aurelien,Jacomy,None,2021-07-25T02:42:03+00:00,None,2021-08-27 18:04:40+00:00,2021-08-29 22:00:07,2021-08-29 19:00:08,2021-08-29,None,True
1,20032,1818,AQAISHKLLSMM,mciriani@byhumane.com,Martin,Ciriani,+5511999813322,2021-07-25T21:00:47+00:00,None,2021-08-19 00:36:14+00:00,2021-08-29 22:00:07,2021-08-29 19:00:08,2021-08-29,None,True
2,20100,1818,AQGJEGFMZPFF,lamarante@byhumane.com,Luana,Amarante,+5585997228378,2021-08-03T11:05:51+00:00,None,2021-08-29 13:25:59+00:00,2021-08-29 22:00:07,2021-08-29 19:00:08,2021-08-29,None,True
3,20118,1818,AQCAGKPUKWUI,contato@hyssopmedia.com,Martin,Ciriani,None,2021-08-06T12:29:27+00:00,2021-08-20T20:01:05+00:00,2021-08-28 01:01:31+00:00,2021-08-29 22:00:07,2021-08-29 19:00:08,2021-08-29,2021-08-20,True
4,20129,1818,AQHQBUDACZMX,jacomya@gmail.com,Aurelien,Jacomy,None,2021-08-08T20:10:50+00:00,None,2021-08-23 12:55:21+00:00,2021-08-29 22:00:07,2021-08-29 19:00:08,2021-08-29,None,True
5,20136,1818,AQNCCPYYWMAN,rodrigo.duarte@danone.com,Rodrigo,Duarte,+5511972511278,2021-08-09T16:50:25+00:00,None,2021-08-13 13:20:30+00:00,2021-08-29 22:00:07,2021-08-29 19:00:08,2021-08-29,None,True
6,20144,1818,AQZWKMAQHJFL,adriano.beltramini@danone.com,Adriano,Beltramini,None,2021-08-10T17:46:06+00:00,None,2021-08-12 13:59:41+00:00,2021-08-29 22:00:07,2021-08-29 19:00:08,2021-08-29,None,True
7,20145,1818,AQOJUJAYOCKL,diego.grippa@danone.com,Diego,Grippa,+5511994338908,2021-08-10T17:46:23+00:00,None,2021-08-13 13:20:29+00:00,2021-08-29 22:00:07,2021-08-29 19:00:08,2021-08-29,None,True
8,20146,1818,AQSNUTGKIPXA,patrick.ferreira@danone.com,Patrick,Ferreira,+5511976764109,2021-08-10T17:46:53+00:00,None,2021-08-11 22:53:52+00:00,2021-08-29 22:00:07,2021-08-29 19:00:08,2021-08-29,None,True
9,20147,1818,AQITGRRWWCVF,marilia.sa@danone.com,Marilia,Sa,+5511997390022,2021-08-10T17:47:45+00:00,None,2021-08-11 22:53:53+00:00,2021-08-29 22:00:07,2021-08-29 19:00:08,2021-08-29,None,True


In [53]:
data = yaml.safe_load(open('../params/non_professional_users.yml','r'))
data

{'non_professionals': {'user_id, type': [[20030, 'Humane'],
   [20032, 'Humane'],
   [20100, 'Humane']]}}

# SCRIPTING

In [ ]:
- user_id:20118,type:'Humane'
- user_id:20129,type:'Humane'
- user_id:20029,type:'Humane'
- user_id:20144,type:'Danone'
- user_id:20145,type:'Danone'
- user_id:20146,type:'Danone'
- user_id:20147,type:'Danone'
- user_id:20172,type:'Danone'
- user_id:20136,type:'Danone'